<a href="https://colab.research.google.com/github/HrithikNambiar/COVID-19-Cough-Detection/blob/main/Audio_Aug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing necessary libraries and data 


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#run the code below to make a copy of the data and copy it into the cloned repo
# %cd /content/drive/MyDrive/InterspeechParalinguisticsChallenge2021/ComParE2021_CCS
# %cp -av dist dist_copy 

In [ ]:
cd /content/gdrive/MyDrive/ComParE2021

/content/gdrive/.shortcut-targets-by-id/1a6R2J7y9rkZWY_SyloMCwmm8qdQAliCz/ComParE2021


In [ ]:
!pip install pydub
!pip install audiomentations
!pip install arff
!pip install -U scikit-learn scipy matplotlib
!pip install -U PyYAML
!pip install audio2numpy
!pip install sounddevice

  Created wheel for arff: filename=arff-0.9-cp37-none-any.whl size=4970 sha256=fd5bf9e96c8be94b8934b1ed7f7ea29c01643f51cb4b15d8fc9cb4979be9092c
  Stored in directory: /root/.cache/pip/wheels/04/d0/70/2c73afedd3ac25c6085b528742c69b9587cbdfa67e5194583b
Successfully built arff
     |████████████████████████████████| 22.3MB 1.5MB/s 
     |████████████████████████████████| 27.4MB 90kB/s 
     |████████████████████████████████| 11.6MB 43.8MB/s 
ERROR: audiomentations 0.16.0 has requirement scipy<1.6.0,>=1.0.0, but you'll have scipy 1.6.1 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
  Found existing installation: matplotlib 3.2.

     |████████████████████████████████| 645kB 4.4MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Created wheel for ffmpeg: filename=ffmpeg-1.4-cp37-none-any.whl size=6083 sha256=cc23771814b60a19d31cfbfc22c1defacc05bbba1ff1c3da33ca4fd53d9c1511
  Stored in directory: /root/.cache/pip/wheels/b6/68/c3/a05a35f647ba871e5572b9bbfc0b95fd1c6637a2219f959e7a
Successfully built ffmpeg


## Run the baseline code

In [ ]:
%%bash
python -m src.svm openXBoW

Fitting 1 folds for each of 20 candidates, totalling 20 fits
UAR: 0.6294398907103825
              precision    recall  f1-score   support

    negative       0.84      0.97      0.90       183
    positive       0.70      0.29      0.41        48

    accuracy                           0.83       231
   macro avg       0.77      0.63      0.66       231
weighted avg       0.81      0.83      0.80       231


Confusion Matrix:

[[177   6]
 [ 34  14]]
Generating test predictions for optimised parameters {'estimator': 'LinearSVC(C=0.01, max_iter=10000, random_state=42)', 'estimator__C': '0.01', 'estimator__class_weight': 'None', 'estimator__loss': 'squared_hinge', 'estimator__max_iter': 10000, 'scaler': 'None'}
Fitting 1 folds for each of 20 candidates, totalling 20 fits
UAR: 0.60724043715847
              precision    recall  f1-score   support

    negative       0.84      0.80      0.82       183
    positive       0.35      0.42      0.38        48

    accuracy                      

In [ ]:
%%bash
python -m src.svm opensmile

Fitting 1 folds for each of 20 candidates, totalling 20 fits
UAR: 0.6139002732240437
              precision    recall  f1-score   support

    negative       0.85      0.75      0.79       183
    positive       0.33      0.48      0.39        48

    accuracy                           0.69       231
   macro avg       0.59      0.61      0.59       231
weighted avg       0.74      0.69      0.71       231


Confusion Matrix:

[[137  46]
 [ 25  23]]
Generating test predictions for optimised parameters {'estimator': "LinearSVC(C=0.1, class_weight='balanced', max_iter=10000, random_state=42)", 'estimator__C': '0.1', 'estimator__class_weight': 'balanced', 'estimator__loss': 'squared_hinge', 'estimator__max_iter': 10000, 'scaler': 'StandardScaler()'}


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

## Augmentation

In [ ]:
import os
import glob 
import shutil

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import librosa
import librosa.display
import soundfile as sf
from pydub import AudioSegment
from audio2numpy import open_audio
from pydub.silence import split_on_silence
from scipy.io.wavfile import read as read_wav
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift,FrequencyMask

from IPython.display import Audio

In [ ]:
fname = "/content/gdrive/MyDrive/ComParE2021/dist/wav/devel_017.wav"
FIG_SIZE = (15,10)

In [ ]:
#Waveform of devel 001
signal, sample_rate = librosa.load(fname, sr=16000)

# WAVEFORM
# display waveform
plt.figure(figsize=FIG_SIZE)
librosa.display.waveplot(signal, sample_rate, alpha=0.4)
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.title("Waveform")

In [ ]:
def split_file_into_chunks(fname, min_silence_len=500, silence_threshold=-40):
  sound_file = AudioSegment.from_wav(fname)
  audio_chunks = split_on_silence(sound_file, min_silence_len=min_silence_len, silence_thresh=silence_threshold)

  for i, chunk in enumerate(audio_chunks):
    out_file = "chunk{0}.wav".format(i)
    # print("exporting", out_file)
    chunk.export(out_file, format="wav")

  return len(audio_chunks)


In [ ]:
os.mkdir('/content/gdrive/MyDrive/ComParE2021/data_2')

os.mkdir('/content/gdrive/MyDrive/ComParE2021/data_2/train/')
os.mkdir('/content/gdrive/MyDrive/ComParE2021/data_2/test/')
os.mkdir('/content/gdrive/MyDrive/ComParE2021/data_2/val/')

In [ ]:
def augment_data(input_dir='/content/drive/MyDrive/ComParE2021/dist/wav', 
                 output_dir='/content/drive/MyDrive/ComParE2021/data/Augmented_train', 
                 csv_file="/content/drive/MyDrive/ComParE2021/dist/lab/train.csv", 
                 csv_output_path="/content/gdrive/MyDrive/ComParE2021/data/train.csv",
                 split_files=True,
                 augmentations=None):
  samplerate = 16000
  df = pd.read_csv(csv_file)

  augmented_fnames = [] 
  augmented_classes = []

  for f in df['filename']: 
    print(f)
    orig_class  = df.loc[df['filename']==f]['label'].item()

    if split_files: 
      audio_chunks = split_file_into_chunks(os.path.join(input_dir, f))

    else:
      sample = librosa.core.load(os.path.join(input_dir, f))[0]
      sf.write("chunk0.wav", sample,samplerate,format='wav')
      audio_chunks = 1

    if "Gaussian_noise" in augmentations:
      for j in range(audio_chunks):
        sample = librosa.core.load(f"chunk{j}.wav")[0]
        augmentar = Compose([AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.010, p=1)])
        augmented_sample = augmentar(sample, sample_rate=16000)
        modified_fname = "gn_"+f[:-4]+"_"+str(j)+".wav"
        sf.write(os.path.join(output_dir, modified_fname), augmented_sample,samplerate,format='wav')

        augmented_fnames.append(modified_fname)
        augmented_classes.append(orig_class)

    if "Shift" in augmentations:
      for j in range(audio_chunks):
        sample = librosa.core.load(f"chunk{j}.wav")[0]
        augmentar = Compose([Shift(min_fraction=-0.1, max_fraction=0.1,rollover=False, p=1)])
        augmented_sample = augmentar(sample, sample_rate=16000)
        modified_fname = "st_"+f[:-4]+"_"+str(j)+".wav"
        sf.write(os.path.join(output_dir, modified_fname), augmented_sample,samplerate,format='wav')

        augmented_fnames.append(modified_fname)
        augmented_classes.append(orig_class)

    if "Frequency_Mask" in augmentations: 
      for j in range(audio_chunks):
        sample = librosa.core.load(f"chunk{j}.wav")[0]
        augmentar = Compose([FrequencyMask(min_frequency_band=0.0, max_frequency_band=0.5, p=0.5)])
        augmented_sample = augmentar(sample, sample_rate=16000)
        modified_fname = "fm_"+f[:-4]+"_"+str(j)+".wav"
        sf.write(os.path.join(output_dir, modified_fname), augmented_sample,samplerate,format='wav')

        augmented_fnames.append(modified_fname)
        augmented_classes.append(orig_class)

    for j in range(audio_chunks):
      sample = librosa.core.load(f"chunk{j}.wav")[0]
      modified_fname = f[:-4]+"_"+str(j)+".wav"
      sf.write(os.path.join(output_dir, modified_fname), sample,samplerate,format='wav')

      augmented_fnames.append(modified_fname)
      augmented_classes.append(orig_class)

  final_df = pd.DataFrame({'filename': augmented_fnames, 'label': augmented_classes})
  final_df.to_csv(csv_output_path, index=False)

In [ ]:
augment_data(input_dir='/content/gdrive/MyDrive/ComParE2021/dist/wav', output_dir='/content/gdrive/MyDrive/ComParE2021/data_2/train/', csv_file="/content/gdrive/MyDrive/ComParE2021/dist/lab/train.csv", csv_output_path='/content/gdrive/MyDrive/ComParE2021/data_2/train.csv' ,split_files=True, augmentations=["Gaussian_noise", "Shift", "Frequency_Mask"])

train_001.wav
train_002.wav
train_003.wav
train_004.wav
train_005.wav
train_006.wav
train_007.wav
train_008.wav
train_009.wav
train_010.wav
train_011.wav
train_012.wav
train_013.wav
train_014.wav
train_015.wav
train_016.wav
train_017.wav
train_018.wav
train_019.wav
train_020.wav
train_021.wav
train_022.wav
train_023.wav
train_024.wav
train_025.wav
train_026.wav
train_027.wav
train_028.wav
train_029.wav
train_030.wav
train_031.wav
train_032.wav
train_033.wav
train_034.wav
train_035.wav
train_036.wav
train_037.wav
train_038.wav
train_039.wav
train_040.wav
train_041.wav
train_042.wav
train_043.wav
train_044.wav
train_045.wav
train_046.wav
train_047.wav
train_048.wav
train_049.wav
train_050.wav
train_051.wav
train_052.wav
train_053.wav
train_054.wav
train_055.wav
train_056.wav
train_057.wav
train_058.wav
train_059.wav
train_060.wav
train_061.wav
train_062.wav
train_063.wav
train_064.wav
train_065.wav
train_066.wav
train_067.wav
train_068.wav
train_069.wav
train_070.wav
train_071.wav
train_

In [ ]:
augment_data(input_dir='/content/gdrive/MyDrive/ComParE2021/dist/wav', output_dir='/content/gdrive/MyDrive/ComParE2021/data_2/test/', csv_file="/content/gdrive/MyDrive/ComParE2021/dist/lab/test.csv", csv_output_path='/content/gdrive/MyDrive/ComParE2021/data_2/test.csv' ,split_files=True, augmentations=["None"])

test_001.wav
test_002.wav
test_003.wav
test_004.wav
test_005.wav
test_006.wav
test_007.wav
test_008.wav
test_009.wav
test_010.wav
test_011.wav
test_012.wav
test_013.wav
test_014.wav
test_015.wav
test_016.wav
test_017.wav
test_018.wav
test_019.wav
test_020.wav
test_021.wav
test_022.wav
test_023.wav
test_024.wav
test_025.wav
test_026.wav
test_027.wav
test_028.wav
test_029.wav
test_030.wav
test_031.wav
test_032.wav
test_033.wav
test_034.wav
test_035.wav
test_036.wav
test_037.wav
test_038.wav
test_039.wav
test_040.wav
test_041.wav
test_042.wav
test_043.wav
test_044.wav
test_045.wav
test_046.wav
test_047.wav
test_048.wav
test_049.wav
test_050.wav
test_051.wav
test_052.wav
test_053.wav
test_054.wav
test_055.wav
test_056.wav
test_057.wav
test_058.wav
test_059.wav
test_060.wav
test_061.wav
test_062.wav
test_063.wav
test_064.wav
test_065.wav
test_066.wav
test_067.wav
test_068.wav
test_069.wav
test_070.wav
test_071.wav
test_072.wav
test_073.wav
test_074.wav
test_075.wav
test_076.wav
test_077.wav

In [ ]:
augment_data(input_dir='/content/gdrive/MyDrive/ComParE2021/dist/wav', output_dir='/content/gdrive/MyDrive/ComParE2021/data_2/val/', csv_file="/content/gdrive/MyDrive/ComParE2021/dist/lab/devel.csv", csv_output_path='/content/gdrive/MyDrive/ComParE2021/data_2/val.csv' ,split_files=True, augmentations=["None"])

devel_001.wav
devel_002.wav
devel_003.wav
devel_004.wav
devel_005.wav
devel_006.wav
devel_007.wav
devel_008.wav
devel_009.wav
devel_010.wav
devel_011.wav
devel_012.wav
devel_013.wav
devel_014.wav
devel_015.wav
devel_016.wav
devel_017.wav
devel_018.wav
devel_019.wav
devel_020.wav
devel_021.wav
devel_022.wav
devel_023.wav
devel_024.wav
devel_025.wav
devel_026.wav
devel_027.wav
devel_028.wav
devel_029.wav
devel_030.wav
devel_031.wav
devel_032.wav
devel_033.wav
devel_034.wav
devel_035.wav
devel_036.wav
devel_037.wav
devel_038.wav
devel_039.wav
devel_040.wav
devel_041.wav
devel_042.wav
devel_043.wav
devel_044.wav
devel_045.wav
devel_046.wav
devel_047.wav
devel_048.wav
devel_049.wav
devel_050.wav
devel_051.wav
devel_052.wav
devel_053.wav
devel_054.wav
devel_055.wav
devel_056.wav
devel_057.wav
devel_058.wav
devel_059.wav
devel_060.wav
devel_061.wav
devel_062.wav
devel_063.wav
devel_064.wav
devel_065.wav
devel_066.wav
devel_067.wav
devel_068.wav
devel_069.wav
devel_070.wav
devel_071.wav
devel_

In [ ]:
# Main Augmentation class

class AudioAugmentation:
    def read_audio_file(self, file_path):  #reading the audio file
        input_length = 50000
        data = librosa.core.load(file_path)[0]
        if len(data) > input_length:
            data1 = data[:input_length]           # split the audio into 2 
            data2 = data[input_length:len(data)]
            #data3 = data[2*input_length : len(data)]
        else:
            data1 = np.pad(data, (0, max(0, input_length - len(data))), "constant")
            data2 = 0
            #data3=0
        return data1,data2

    def write_audio_file(self, file, data, sample_rate=16000): #writing the augmented file into a new file name 
        librosa.output.write_wav(file, data, sample_rate)

    def plot_time_series(self, data):   #plotting the augmented waveform
        fig = plt.figure(figsize=(14, 8))
        plt.title('Raw wave ')
        plt.ylabel('Amplitude')
        plt.plot(np.linspace(0, 1, len(data)), data)
        plt.show()

    def add_noise(self, data):   #adding a uniform noise
        noise = np.random.randn(len(data))
        data_noise = data + 0.005 * noise
        return data_noise

    def shift(self, data):
        return np.roll(data, 1600)

    def stretch(self, data, rate=1):
        input_length = 16000
        data = librosa.effects.time_stretch(data, rate)
        if len(data) > input_length:
            data = data[:input_length]
        else: 
            data = np.pad(data, (0, max(0, input_length - len(data))), "constant")
        return data

    # We can use the function below for augmentation
      
    def augment(self,data):   # Apart from the function coded above , augmentation using Audiomentation
        augment = Compose([AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.010, p=1),  # ADD gaussian noice of a small amplitude
                           #TimeStretch(min_rate=0.8, max_rate=1.25, p=0.5),
                           #PitchShift(min_semitones=-4, max_semitones=4, p=0.5), not changing the pitch because the pitch of cough might have information for the model
                           Shift(min_fraction=-0.1, max_fraction=0.1,rollover=False, p=1), #shifting the audio
                           FrequencyMask(min_frequency_band=0.0, max_frequency_band=0.5, p=0.5), #frequency mask with a probability 0.5
                           ])
        augmented_samples = augment(data, sample_rate=16000)
        return augmented_samples


In [ ]:
aa = AudioAugmentation()
samplerate=16000  # All the data provided to us has a sample_rate of 16000 , hence the augmented file should have the same
 
# Read and show sound
data = aa.read_audio_file("/content/drive/MyDrive/ComParE2021/dist/wav/train_001.wav")
 
data1 = data[0]
data2 = data[1]
#data3 = data[2]
 
# Adding uniform noise to sound
# data_noise = aa.add_noise(data[0])
# aa.plot_time_series(data_noise)
 
# Shifting the sound
# data_roll = aa.shift(data_noise)
# aa.plot_time_series(data_roll)
 
# Stretching the sound
# data_stretch = aa.stretch(data[0], 0.8)
# aa.plot_time_series(data_stretch)
 
# Augmentation using audioAugment library
 
data_aug1=aa.augment(data1)
#aa.plot_time_series(data_aug1)
 
data_aug2=aa.augment(data2)
#aa.plot_time_series(data_aug2)
 
#data_aug3=aa.augment(data3)
 
sf.write('/content/drive/MyDrive/ComParE2021/output/val1_cough1.wav', data1,samplerate,format='wav')
sf.write('/content/drive/MyDrive/ComParE2021/output/val1_cough2.wav', data2,samplerate,format='wav')

sf.write('/content/drive/MyDrive/ComParE2021/output/val1_aug_cough1.wav', data_aug1,samplerate,format='wav')
sf.write('/content/drive/MyDrive/ComParE2021/output/val1_aug_cough2.wav', data_aug2,samplerate,format='wav')


In [ ]:
#Augmenting and writing into new folder

input_directory = '/content/drive/MyDrive/ComParE2021/dist/wav'
output_directory = '/content/drive/MyDrive/ComParE2021/data/Augmented_train'
i=1
for file in sorted(glob.glob('/content/drive/MyDrive/ComParE2021/dist/wav/train_***.wav')):

  aa = AudioAugmentation()
  samplerate=16000  
  data = aa.read_audio_file(file)
  data1 = data[0]
  data2 = data[1]
 
  data_aug1=aa.augment(data1)
  data_aug2=aa.augment(data2)
  
  sf.write(f'/content/drive/MyDrive/ComParE2021/data/Augmented_train/train_{i:0>3d}_cough1.wav', data1,samplerate,format='wav')
  sf.write(f'/content/drive/MyDrive/ComParE2021/data/Augmented_train/train_{i:0>3d}_cough2.wav', data2,samplerate,format='wav')
  # sf.write(f'/content/drive/MyDrive/ComParE2021/data/Augmented_train/train_{i:0>3d}_cough3.wav', data3,samplerate,format='wav')
  sf.write(f'/content/drive/MyDrive/ComParE2021/data/Augmented_train/train_{i:0>3d}_aug_cough1.wav', data_aug1,samplerate,format='wav')
  sf.write(f'/content/drive/MyDrive/ComParE2021/data/Augmented_train/train_{i:0>3d}_aug_cough2.wav', data_aug2,samplerate,format='wav')
  # sf.write(f'/content/drive/MyDrive/ComParE2021/data/Augmented_train/train_{i:0>3d}_aug_cough3.wav', data_aug3,samplerate,format='wav')
  i=i+1


In [ ]:
# audio of devel 001
from IPython.display import Audio
Audio('/content/drive/MyDrive/ComParE2021/output/val1_aug_cough2.wav', rate=16000)  #the val 1 data provided to us